In [1]:
#Import relevant libraries
import pandas as pd
import statsmodels.formula.api as sm
import statsmodels.stats.descriptivestats as dstats
import numpy as np  # For Data Manipulation
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder  # For Encoding Categorical Data [Nominal | Ordinal]
from sklearn.preprocessing import OneHotEncoder  # For Creating Dummy Variables of Categorical Data [Nominal]
from sklearn.impute import SimpleImputer, KNNImputer  # For Imputation of Missing Data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler  # For Rescaling Data
from sklearn.model_selection import train_test_split  # For Splitting Data into Training & Testing Sets
import matplotlib.pyplot as plt
import seaborn as sns  # For Data Visualization
import scipy.cluster.hierarchy as sch  # For Hierarchical Clustering
from sklearn.cluster import AgglomerativeClustering as agclus, KMeans as kmclus  # For Agglomerative & K-Means Clustering
from sklearn.metrics import silhouette_score as sscore, davies_bouldin_score as dbscore  # For Clustering Model Evaluation
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report # For Decision Tree Model Evaluation
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree  # For Decision Tree Model
import scipy.stats as stats
import statsmodels.stats.multicomp as multi

In [2]:
import pandas as pd
import requests
from io import StringIO

# Replace with your actual file ID
file_id = '1GnKJ7ObMqgTWZI27ySgdWCuy7bcYD77-'

# Construct the URL
url = f'https://drive.google.com/uc?id={file_id}'

# Fetch the data using requests
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad responses

# Read the data into a pandas DataFrame using StringIO
sd48_df = pd.read_csv(StringIO(response.text))

# Display the first few rows of the DataFrame
sd48_df.head()

,Transaction_ID,Country,Product,Import_Export,Quantity,Value,Date,Category,Port,Customs_Code,Weight,Shipping_Method,Supplier,Customer,Invoice_Number,Payment_Terms
0,e3e70682-c209-4cac-a29f-6fbed82c07cd,Colombia,describe,Export,1979,9506.57,07-12-2023,Machinery,Robertbury,620537,4248.65,Air,"Garrison, Hubbard and Hendricks",Seth Hall,21000294,Cash on Delivery
1,f728b4fa-4248-4e3a-8a5d-2f346baa9455,Chile,president,Export,5763,7100.91,04-04-2023,Clothing,Shahport,927600,4340.81,Air,Webb-Mack,Kimberly Ryan,88738432,Prepaid
2,eb1167b3-67a9-4378-bc65-c1e582e2e662,Turkey,far,Import,5740,2450.34,21-08-2024,Electronics,South Joshuatown,299258,4538.41,Air,"Mendez, Jones and Johnson",Ryan Silva,89922099,Prepaid
3,f7c1bd87-4da5-4709-9471-3d60c8a70639,Christmas Island,agency,Export,2592,7226.42,09-05-2022,Furniture,Adamfort,996084,4886.14,Air,Schroeder-Smith,Jacob Gray,63216265,Net 60
4,e443df78-9558-467f-9ba9-1faf7a024204,Finland,policy,Export,2622,2817.29,03-04-2023,Electronics,Juliebury,555981,4406.74,Air,Zimmerman LLC,Amy Stephens,94600248,Cash on Delivery


In [3]:
# Info for dataset
sd48_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Transaction_ID   15000 non-null  object 
 1   Country          15000 non-null  object 
 2   Product          15000 non-null  object 
 3   Import_Export    15000 non-null  object 
 4   Quantity         15000 non-null  int64  
 5   Value            15000 non-null  float64
 6   Date             15000 non-null  object 
 7   Category         15000 non-null  object 
 8   Port             15000 non-null  object 
 9   Customs_Code     15000 non-null  int64  
 10  Weight           15000 non-null  float64
 11  Shipping_Method  15000 non-null  object 
 12  Supplier         15000 non-null  object 
 13  Customer         15000 non-null  object 
 14  Invoice_Number   15000 non-null  int64  
 15  Payment_Terms    15000 non-null  object 
dtypes: float64(2), int64(3), object(11)
memory usage: 1.8+ MB


In [4]:
# Create sample of 5001
sd48_sample = sd48_df.sample(n= 5001, random_state=55048)
sd48_sample

,Transaction_ID,Country,Product,Import_Export,Quantity,Value,Date,Category,Port,Customs_Code,Weight,Shipping_Method,Supplier,Customer,Invoice_Number,Payment_Terms
8213,d2d4cbf0-9752-4ae7-ba2c-12f7ac93c4d9,Netherlands Antilles,help,Import,3079,110.75,27-03-2023,Furniture,North Tanyachester,728366,1117.96,Sea,"Mathews, Butler and Davenport",Steven Clark,22670612,Net 30
10884,c5593281-445d-4c07-9ca6-d9bf3fa4cb44,Senegal,music,Import,2088,2780.03,28-09-2023,Machinery,Elizabethchester,373176,4460.68,Sea,Williamson Group,Kevin Chapman,23118544,Prepaid
13183,ccd441c3-f868-478f-9e5a-9bdfea9b014f,Thailand,often,Export,9857,8274.73,03-10-2020,Toys,South Natasha,414404,51.16,Sea,"Johnson, Wang and Ritter",Sean Gonzalez,14921248,Prepaid
1346,84ddf869-b816-4f1a-9677-013ba8e19b83,Switzerland,tough,Export,8067,7866.85,10-04-2020,Machinery,New Scott,182712,3143.58,Land,Marks PLC,Jeffrey Wilcox,13685688,Net 60
8392,0b10c800-2b07-4d36-aa6e-df80fb25a0f9,Mali,church,Import,9673,436.62,25-10-2020,Toys,Port Carolport,472557,1160.50,Land,"Hall, Jackson and Johnston",Ann Miller,10291630,Net 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11985,e69ece35-c159-4644-9928-42721631b516,United Kingdom,fight,Export,1684,7073.68,25-06-2023,Machinery,Lake Michael,685168,841.08,Sea,"Garcia, Mooney and Christian",Kimberly Cook,62085317,Net 60
471,f1c58f44-7e08-4c1b-8da6-a326451437d6,Bermuda,next,Import,3355,2626.52,27-10-2020,Machinery,Jessicafurt,129999,4238.32,Air,"Little, Barrett and Larson",Kelly Carter,24473327,Net 60
9869,f8a5392c-7209-43e5-8e9a-e77c674447d3,Paraguay,minute,Import,6959,6239.48,31-05-2023,Machinery,Loriside,412206,4565.55,Land,White Group,Veronica Collier,67370214,Net 30
11481,2805f980-c013-4085-babc-8a4acd03fc4b,Monaco,right,Export,8201,555.06,20-12-2021,Furniture,Curtisfurt,901175,2215.64,Land,Silva-Juarez,Collin Leonard,31417209,Cash on Delivery


In [5]:
# Missing info
sd48_sample.info() # Dataframe Information (Provide Information on Missing Data)
sd48_variable_missing_data = sd48_sample.isna().sum()
sd48_variable_missing_per = (sd48_sample.isna().sum()/len(sd48_df))*100
sd48_variable_missing_info = pd.DataFrame({'Total Missing Values' : sd48_variable_missing_data,
                              'Percentage' : sd48_variable_missing_per})
print(sd48_variable_missing_info) # Variable-wise Missing Data Information
sd48_record_missing_data = sd48_sample.isna().sum(axis=1).sort_values(ascending=False)
print(sd48_record_missing_data) # Record-wise Missing Data Information in Descending Order

<class 'pandas.core.frame.DataFrame'>
Index: 5001 entries, 8213 to 2772
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Transaction_ID   5001 non-null   object 
 1   Country          5001 non-null   object 
 2   Product          5001 non-null   object 
 3   Import_Export    5001 non-null   object 
 4   Quantity         5001 non-null   int64  
 5   Value            5001 non-null   float64
 6   Date             5001 non-null   object 
 7   Category         5001 non-null   object 
 8   Port             5001 non-null   object 
 9   Customs_Code     5001 non-null   int64  
 10  Weight           5001 non-null   float64
 11  Shipping_Method  5001 non-null   object 
 12  Supplier         5001 non-null   object 
 13  Customer         5001 non-null   object 
 14  Invoice_Number   5001 non-null   int64  
 15  Payment_Terms    5001 non-null   object 
dtypes: float64(2), int64(3), object(11)
memory usage: 793.2+ KB
  

In [6]:
# Calculate variable-wise missing data percentage
variable_missing_percentage = sd48_sample.isnull().sum() * 100 / len(sd48_sample)

# Calculate record-wise missing data percentage
record_missing_percentage = sd48_sample.isnull().sum(axis=1) * 100 / len(sd48_sample.columns)

# Print variable-wise and record-wise missing data information
print("Variable-wise missing data percentage:\n", variable_missing_percentage)
print("\nRecord-wise missing data percentage:\n", record_missing_percentage)

# Drop or impute based on variable missing data percentage
for col in sd48_sample.columns:
  if variable_missing_percentage[col] >= 50:
    # Drop the variable
    sd48_sample = sd48_sample.drop(col, axis=1)
  else:
    # Impute missing values (using mean for numerical, mode for categorical)
    if pd.api.types.is_numeric_dtype(sd48_sample[col]):
      sd48_sample[col] = sd48_sample[col].fillna(sd48_sample[col].mean())
    else:
      sd48_sample[col] = sd48_sample[col].fillna(sd48_sample[col].mode()[0])


# Drop or impute based on record missing data percentage
for index in sd48_sample.index:
  if record_missing_percentage[index] >= 50:
    # Drop the record
    sd48_sample = sd48_sample.drop(index)
  else:
    # Impute missing values (using mean for numerical, mode for categorical)
    for col in sd48_sample.columns:
      if pd.isna(sd48_sample.loc[index,col]):
        if pd.api.types.is_numeric_dtype(sd48_sample[col]):
          sd48_sample.loc[index, col] = sd48_sample[col].mean()
        else:
          sd48_sample.sample[index,col] = sd48_sample[col].mode()[0]

Variable-wise missing data percentage:
 Transaction_ID     0.0
Country            0.0
Product            0.0
Import_Export      0.0
Quantity           0.0
Value              0.0
Date               0.0
Category           0.0
Port               0.0
Customs_Code       0.0
Weight             0.0
Shipping_Method    0.0
Supplier           0.0
Customer           0.0
Invoice_Number     0.0
Payment_Terms      0.0
dtype: float64

Record-wise missing data percentage:
 8213     0.0
10884    0.0
13183    0.0
1346     0.0
8392     0.0
        ... 
11985    0.0
471      0.0
9869     0.0
11481    0.0
2772     0.0
Length: 5001, dtype: float64


In [7]:
sd48_sample.columns

Index(['Transaction_ID', 'Country', 'Product', 'Import_Export', 'Quantity',
       'Value', 'Date', 'Category', 'Port', 'Customs_Code', 'Weight',
       'Shipping_Method', 'Supplier', 'Customer', 'Invoice_Number',
       'Payment_Terms'],
      dtype='object')

In [8]:
# Segregate categorical and non categorical variables
index_cols = ["Transaction_ID"]

categorical_cols = ["Country", "Product", "Import_Export", "Date", "Category", "Port",
                     "Customs_Code", "Shipping_Method", "Supplier", "Customer", "Invoice_Number", "Payment_Terms"]

non_categorical_cols = [ "Quantity", "Value", "Weight"]

# Removing duplicates in case of overlap
categorical_cols = list(set(categorical_cols))
non_categorical_cols = list(set(non_categorical_cols))

sd48_sample_non_cat = sd48_sample[non_categorical_cols]
sd48_sample_cat = sd48_sample[categorical_cols]

print("Categorical Columns:", categorical_cols,)
print("Non-Categorical Columns:", non_categorical_cols)

Categorical Columns: ['Customer', 'Shipping_Method', 'Customs_Code', 'Product', 'Country', 'Import_Export', 'Invoice_Number', 'Supplier', 'Date', 'Payment_Terms', 'Category', 'Port']
Non-Categorical Columns: ['Weight', 'Quantity', 'Value']


In [9]:
sd48_sample_non_cat

,Weight,Quantity,Value
8213,1117.96,3079,110.75
10884,4460.68,2088,2780.03
13183,51.16,9857,8274.73
1346,3143.58,8067,7866.85
8392,1160.50,9673,436.62
...,...,...,...
11985,841.08,1684,7073.68
471,4238.32,3355,2626.52
9869,4565.55,6959,6239.48
11481,2215.64,8201,555.06


In [10]:
sd48_sample_cat

,Customer,Shipping_Method,Customs_Code,Product,Country,Import_Export,Invoice_Number,Supplier,Date,Payment_Terms,Category,Port
8213,Steven Clark,Sea,728366,help,Netherlands Antilles,Import,22670612,"Mathews, Butler and Davenport",27-03-2023,Net 30,Furniture,North Tanyachester
10884,Kevin Chapman,Sea,373176,music,Senegal,Import,23118544,Williamson Group,28-09-2023,Prepaid,Machinery,Elizabethchester
13183,Sean Gonzalez,Sea,414404,often,Thailand,Export,14921248,"Johnson, Wang and Ritter",03-10-2020,Prepaid,Toys,South Natasha
1346,Jeffrey Wilcox,Land,182712,tough,Switzerland,Export,13685688,Marks PLC,10-04-2020,Net 60,Machinery,New Scott
8392,Ann Miller,Land,472557,church,Mali,Import,10291630,"Hall, Jackson and Johnston",25-10-2020,Net 30,Toys,Port Carolport
...,...,...,...,...,...,...,...,...,...,...,...,...
11985,Kimberly Cook,Sea,685168,fight,United Kingdom,Export,62085317,"Garcia, Mooney and Christian",25-06-2023,Net 60,Machinery,Lake Michael
471,Kelly Carter,Air,129999,next,Bermuda,Import,24473327,"Little, Barrett and Larson",27-10-2020,Net 60,Machinery,Jessicafurt
9869,Veronica Collier,Land,412206,minute,Paraguay,Import,67370214,White Group,31-05-2023,Net 30,Machinery,Loriside
11481,Collin Leonard,Land,901175,right,Monaco,Export,31417209,Silva-Juarez,20-12-2021,Cash on Delivery,Furniture,Curtisfurt


In [11]:
# Initialize the OrdinalEncoder
sd48_oe = OrdinalEncoder()

# Apply Ordinal Encoding to the categorical columns
sd48_sample_encoded = sd48_sample.copy()
sd48_sample_encoded[categorical_cols] = sd48_oe.fit_transform(sd48_sample[categorical_cols])

# Show the result
print(sd48_sample_encoded)

                             Transaction_ID  Country  Product  Import_Export  \
8213   d2d4cbf0-9752-4ae7-ba2c-12f7ac93c4d9    152.0    371.0            1.0   
10884  c5593281-445d-4c07-9ca6-d9bf3fa4cb44    192.0    526.0            1.0   
13183  ccd441c3-f868-478f-9e5a-9bdfea9b014f    215.0    564.0            0.0   
1346   84ddf869-b816-4f1a-9677-013ba8e19b83    210.0    875.0            0.0   
8392   0b10c800-2b07-4d36-aa6e-df80fb25a0f9    131.0    161.0            1.0   
...                                     ...      ...      ...            ...   
11985  e69ece35-c159-4644-9928-42721631b516    229.0    300.0            0.0   
471    f1c58f44-7e08-4c1b-8da6-a326451437d6     23.0    544.0            1.0   
9869   f8a5392c-7209-43e5-8e9a-e77c674447d3    169.0    509.0            1.0   
11481  2805f980-c013-4085-babc-8a4acd03fc4b    141.0    702.0            0.0   
2772   b18f3aee-34a2-4626-804b-d39754a9d898    124.0    899.0            0.0   

       Quantity    Value    Date  Categ

In [20]:
# Display encoding mapping

for col in categorical_cols:
    print(f"\nEncoding Mapping for {col}:")
    for i, category in enumerate(sd48_oe.categories_[categorical_cols.index(col)]):
        print(f"{category} -> {i}")

Streaming output truncated to the last 5000 lines.
19-01-2024 -> 1017
19-02-2020 -> 1018
19-02-2021 -> 1019
19-02-2023 -> 1020
19-03-2020 -> 1021
19-03-2021 -> 1022
19-03-2022 -> 1023
19-03-2023 -> 1024
19-03-2024 -> 1025
19-04-2020 -> 1026
19-04-2021 -> 1027
19-04-2022 -> 1028
19-04-2023 -> 1029
19-04-2024 -> 1030
19-05-2020 -> 1031
19-05-2021 -> 1032
19-05-2022 -> 1033
19-05-2023 -> 1034
19-06-2020 -> 1035
19-06-2022 -> 1036
19-06-2023 -> 1037
19-07-2020 -> 1038
19-07-2021 -> 1039
19-07-2023 -> 1040
19-07-2024 -> 1041
19-08-2020 -> 1042
19-08-2021 -> 1043
19-08-2022 -> 1044
19-08-2023 -> 1045
19-08-2024 -> 1046
19-09-2019 -> 1047
19-09-2020 -> 1048
19-09-2021 -> 1049
19-09-2022 -> 1050
19-09-2023 -> 1051
19-10-2020 -> 1052
19-10-2021 -> 1053
19-10-2022 -> 1054
19-10-2023 -> 1055
19-11-2019 -> 1056
19-11-2020 -> 1057
19-11-2021 -> 1058
19-11-2022 -> 1059
19-11-2023 -> 1060
19-12-2019 -> 1061
19-12-2020 -> 1062
19-12-2021 -> 1063
19-12-2022 -> 1064
19-12-2023 -> 1065
20-01-2020 -> 1066

In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply min-max scaling
sd48_sample_encoded[non_categorical_cols] = scaler.fit_transform(sd48_sample_encoded[non_categorical_cols])

# Convert back to DataFrame
sd48_sample_encoded[non_categorical_cols] = pd.DataFrame(sd48_sample_encoded[non_categorical_cols], columns=sd48_sample_non_cat.columns, index=sd48_sample_non_cat.index)

sd48_sample_encoded[non_categorical_cols]

,Weight,Quantity,Value
8213,0.223193,0.307677,0.000796
10884,0.892093,0.208488,0.270522
13183,0.009719,0.986087,0.825752
1346,0.628532,0.806926,0.784537
8392,0.231705,0.967671,0.033725
...,...,...,...
11985,0.167787,0.168051,0.704388
471,0.847597,0.335302,0.255010
9869,0.913078,0.696026,0.620094
11481,0.442846,0.820338,0.045693


In [22]:
sd48_sample_encoded

,Transaction_ID,Country,Product,Import_Export,Quantity,Value,Date,Category,Port,Customs_Code,Weight,Shipping_Method,Supplier,Customer,Invoice_Number,Payment_Terms
8213,d2d4cbf0-9752-4ae7-ba2c-12f7ac93c4d9,152.0,371.0,1.0,0.307677,0.000796,1475.0,2.0,2738.0,3481.0,0.223193,2.0,2483.0,4361.0,1108.0,1.0
10884,c5593281-445d-4c07-9ca6-d9bf3fa4cb44,192.0,526.0,1.0,0.208488,0.270522,1560.0,3.0,916.0,1539.0,0.892093,2.0,4438.0,2697.0,1130.0,3.0
13183,ccd441c3-f868-478f-9e5a-9bdfea9b014f,215.0,564.0,0.0,0.986087,0.825752,152.0,4.0,3658.0,1763.0,0.009719,2.0,1959.0,4165.0,763.0,3.0
1346,84ddf869-b816-4f1a-9677-013ba8e19b83,210.0,875.0,0.0,0.806926,0.784537,520.0,3.0,2384.0,455.0,0.628532,1.0,2376.0,2018.0,703.0,2.0
8392,0b10c800-2b07-4d36-aa6e-df80fb25a0f9,131.0,161.0,1.0,0.967671,0.033725,1395.0,4.0,2911.0,2107.0,0.231705,1.0,1497.0,315.0,519.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11985,e69ece35-c159-4644-9928-42721631b516,229.0,300.0,0.0,0.168051,0.704388,1380.0,3.0,1712.0,3254.0,0.167787,2.0,1254.0,2730.0,3082.0,2.0
471,f1c58f44-7e08-4c1b-8da6-a326451437d6,23.0,544.0,1.0,0.335302,0.255010,1507.0,3.0,1289.0,164.0,0.847597,0.0,2267.0,2634.0,1202.0,2.0
9869,f8a5392c-7209-43e5-8e9a-e77c674447d3,169.0,509.0,1.0,0.696026,0.620094,1693.0,3.0,1897.0,1743.0,0.913078,1.0,4330.0,4705.0,3344.0,1.0
11481,2805f980-c013-4085-babc-8a4acd03fc4b,141.0,702.0,0.0,0.820338,0.045693,1121.0,2.0,499.0,4427.0,0.442846,1.0,3681.0,960.0,1543.0,0.0
